<a href="https://colab.research.google.com/github/aditya-vashisht-111/BookWise-KNN-Book-Recommender/blob/main/Book_Recommendation_Engine_using_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2026-01-21 04:34:47--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   128MB/s    in 0.2s    

2026-01-21 04:34:47 (128 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
# 1. Filter Users: Remove users with less than 200 ratings
user_counts = df_ratings['user'].value_counts()
active_users = user_counts[user_counts >= 200].index
df_ratings_filtered = df_ratings[df_ratings['user'].isin(active_users)]

print(f"Filtered users. Remaining ratings: {len(df_ratings_filtered)}")

Filtered users. Remaining ratings: 527556


In [5]:
# 2. Filter Books: Remove books with less than 100 ratings
book_counts = df_ratings['isbn'].value_counts()
popular_books = book_counts[book_counts >= 100].index

print(f"Identify popular books. Count: {len(popular_books)}")

Identify popular books. Count: 731


In [6]:
# 3. Create Final DataFrame and Merge
df = df_ratings_filtered[df_ratings_filtered['isbn'].isin(popular_books)]
df = pd.merge(df, df_books, on='isbn')

print(f"Merged dataframe shape: {df.shape}")

Merged dataframe shape: (49517, 5)


In [7]:
# 4. Remove Duplicates & Create Pivot Table
df = df.drop_duplicates(['title', 'user'])
book_pivot = df.pivot(index='title', columns='user', values='rating').fillna(0)

# 5. Convert to CSR Matrix
book_matrix = csr_matrix(book_pivot.values)

print("Data processing complete. Matrix shape:", book_matrix.shape)

Data processing complete. Matrix shape: (673, 888)


In [8]:
# 6. Fit the Model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(book_matrix)

print("Model trained successfully.")

Model trained successfully.


In [15]:
# Function to return recommended books
def get_recommends(book=""):
    # 1. Check if the book exists in our database
    if book not in book_pivot.index:
        print(f"Book '{book}' not found in database.")
        return [book, []]

    # 2. Get the numeric vector for the book
    book_loc = book_pivot.index.get_loc(book)
    book_vector = book_pivot.iloc[book_loc, :].values.reshape(1, -1)

    # 3. Find 6 nearest neighbors (the book itself is #1, so we need 5 more)
    distances, indices = model_knn.kneighbors(book_vector, n_neighbors=6)

    recommended_books = []
    neighbor_list = []

    # 4. Loop through results (skip index 0, which is the input book itself)
    for i in range(1, 6):
        title = book_pivot.index[indices[0][i]]
        dist = distances[0][i]
        neighbor_list.append([title, dist])

    # 5. Sort by distance descending (farthest first) for the test
    neighbor_list.sort(key=lambda x: x[1], reverse=True)

    # 6. Final format
    recommended_books = [book, neighbor_list]

    return recommended_books

In [16]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", np.float32(0.8016211)], ['The Weight of Water', np.float32(0.77085835)], ['The Surgeon', np.float32(0.7699411)], ['I Know This Much Is True', np.float32(0.7677075)], ['The Lovely Bones: A Novel', np.float32(0.7234864)]]]
You passed the challenge! 🎉🎉🎉🎉🎉
